In [1]:
import pandas as pd
import numpy as np
from path import Path
import torch.utils.data as data
from imageio import imread
import os
import torch
import torch.nn as nn
import torchvision.models as models
import torch.nn.functional as F 
import matplotlib.pyplot as plt
%matplotlib inline 
from torch.utils.data.sampler import SubsetRandomSampler

from PIL import *
import ast

In [2]:
import numpy as np

class ScaledDotProductAttention(nn.Module):
    ''' Scaled Dot-Product Attention '''
    # given query, key,value it finds the rightful weighted component of v to get the attention applied ouput
    #q,v,k- batch X length of sequence X features or encoding
    #attention sholuld be -batchX7X7
    def __init__(self, temperature, attn_dropout=0.1):
        super().__init__()
        self.temperature = temperature
        self.dropout = nn.Dropout(attn_dropout)
        self.softmax = nn.Softmax(dim=2)

    def forward(self, q, k, v, mask=None):
#         print(k.transpose(1,2).shape)

        attn = torch.bmm(q, k.transpose(1, 2)) 
#         print(attn.shape)
        attn = attn / self.temperature

        if mask is not None:
            attn = attn.masked_fill(mask, -np.inf)

        attn = self.softmax(attn)
        attn = self.dropout(attn)
#         print(str(attn.shape)+" "+str(v.shape))
        output = torch.bmm(attn, v)

        return output, attn
SDP=ScaledDotProductAttention(5)
Ss=SDP(torch.zeros(5,6,100),torch.zeros(5,6,100),torch.zeros(5,6,100))
# print(Ss[0].shape)
class MultiHeadAttention(nn.Module):
    ''' Multi-Head Attention module '''

    def __init__(self, n_head, d_model, d_k, d_v, dropout=0.1):
        super().__init__()

        self.n_head = n_head
        self.d_k = d_k
        self.d_v = d_v

        self.w_qs = nn.Linear(d_model, n_head * d_k)
        self.w_ks = nn.Linear(d_model, n_head * d_k)
        self.w_vs = nn.Linear(d_model, n_head * d_v)
        nn.init.normal_(self.w_qs.weight, mean=0, std=np.sqrt(2.0 / (d_model + d_k)))
        nn.init.normal_(self.w_ks.weight, mean=0, std=np.sqrt(2.0 / (d_model + d_k)))
        nn.init.normal_(self.w_vs.weight, mean=0, std=np.sqrt(2.0 / (d_model + d_v)))

        self.attention = ScaledDotProductAttention(temperature=np.power(d_k, 0.5))
        self.layer_norm = nn.LayerNorm(d_model)

        self.fc = nn.Linear(n_head * d_v, d_model)
        nn.init.xavier_normal_(self.fc.weight)

        self.dropout = nn.Dropout(dropout)


    def forward(self, q, k, v, mask=None):

        d_k, d_v, n_head = self.d_k, self.d_v, self.n_head

        sz_b, len_q, _ = q.size()
        sz_b, len_k, _ = k.size()
        sz_b, len_v, _ = v.size()
#         print(str(sz_b)+"die")

        residual = q

        q = self.w_qs(q).view(sz_b, len_q, n_head, d_k)
        k = self.w_ks(k).view(sz_b, len_k, n_head, d_k)
        v = self.w_vs(v).view(sz_b, len_v, n_head, d_v)

        q = q.permute(2, 0, 1, 3).contiguous().view(-1, len_q, d_k) # (n*b) x lq x dk
        k = k.permute(2, 0, 1, 3).contiguous().view(-1, len_k, d_k) # (n*b) x lk x dk
        v = v.permute(2, 0, 1, 3).contiguous().view(-1, len_v, d_v) # (n*b) x lv x dv
#         print("v-"+str(v.shape))
        if mask is not None:
            mask = mask.repeat(n_head, 1, 1) # (n*b) x .. x ..
        output, attn = self.attention(q, k, v, mask=mask)
#         print(q.shape,k.shape,v.shape)
        output = output.view(n_head, sz_b, len_q, d_v)
        output = output.permute(1, 2, 0, 3).contiguous().view(sz_b, len_q, -1) # b x lq x (n*dv)

        output = self.dropout(self.fc(output))
        output = self.layer_norm(output + residual)

        return output, attn
MHA=MultiHeadAttention(4,15,15,15)
op=MHA(torch.zeros(5,7,15),torch.zeros(5,7,15),torch.zeros(5,7,15))
# print(op[0].shape)
class PositionwiseFeedForward(nn.Module):
    ''' A two-feed-forward-layer module '''

    def __init__(self, d_in, d_hid, dropout=0.1):
        super().__init__()
        self.w_1 = nn.Conv1d(d_in, d_hid, 1) # position-wise
        self.w_2 = nn.Conv1d(d_hid, d_in, 1) # position-wise
        self.layer_norm = nn.LayerNorm(d_in)
        self.dropout = nn.Dropout(dropout)

    def forward(self, x):
        residual = x
        output = x.transpose(1, 2)
#         print("FCC-"+str(output.shape))
#         print("FFC_out-"+str(self.w_1(output).shape))
        output = self.w_2(F.relu(self.w_1(output)))
        output = output.transpose(1, 2)
        output = self.dropout(output)
        output = self.layer_norm(output + residual)
        return output

    

In [25]:
class EncoderLayer(nn.Module):
    ''' Compose with two layers '''

    def __init__(self, d_model, d_inner, n_head, d_k, d_v, dropout=0.1):
        super(EncoderLayer, self).__init__()
        self.slf_attn = MultiHeadAttention(
            n_head, d_model, d_k, d_v, dropout=dropout)
        self.pos_ffn = PositionwiseFeedForward(d_model, d_inner, dropout=dropout)

    def forward(self, enc_input, non_pad_mask=None, slf_attn_mask=None):
        enc_output, enc_slf_attn = self.slf_attn(
            enc_input, enc_input, enc_input, mask=slf_attn_mask)
#         print("ENC_o")
#         print(enc_output.shape)


        enc_output = self.pos_ffn(enc_output)


        return enc_output, enc_slf_attn
    
XX=EncoderLayer(15,10,4,10,10)

zz=XX(torch.zeros(5,7,15))
# print("ENc")
# print(zz[0].shape)
# print("start")
class Encoder(nn.Module):
    ''' A encoder model with self attention mechanism. '''

    def __init__(self,n_modality,d_model,n_head,d_k,d_v,dropout,n_layers,d_inner=500):
        #d_model - number of features in input 100 here
        #n_head - number of heads of multihaded attention
        #d_k=d_q=  number of features in query, key
        #d_v = number of features in value whose weighted(attentioned) sum we gonna take
        

        super().__init__()
        self.n_modality=n_modality
#         self.stn=nn.ModuleList([SpatialTransformer(3, (240,240), 8) for _ in range(n_ref)])
        
        self.layer_stack = nn.ModuleList([EncoderLayer(d_model, d_inner, n_head, d_k, d_v, dropout=dropout) 
                                          for _ in range(n_layers)])
#         self.em=nn.Linear(225,100)
        self.fc1=nn.Linear(d_model*n_modality,300)
        self.relu=nn.ReLU()
        self.fc2=nn.Linear(300,100)
        self.fc3=nn.Linear(100,2)
#         self.fc4=nn.Linear(50,3)
        self.bn1 = nn.BatchNorm1d(num_features=300)
        self.bn2 = nn.BatchNorm1d(num_features=100)
        self.softmax=nn.Softmax(1)
        self.dropout = nn.Dropout(dropout)

    def forward(self, embeddings1,embeddings2 ):


        
        encodings_total=[embeddings1,embeddings2]

        enc_output=torch.stack(encodings_total,0)

        
        enc_output=enc_output.permute(1,0,2)
#         print("encoding_OUTPUT2-"+str(enc_output.shape))

        for enc_layer in self.layer_stack:
            enc_output, enc_slf_attn = enc_layer(enc_output,non_pad_mask=None,slf_attn_mask=None)
           
        
        final_input=enc_output.reshape(enc_output.shape[0],-1)

        final=self.relu(self.fc3(self.bn2(self.relu((self.fc2(self.bn1(self.relu(self.fc1(final_input)))))))))
        
        return(final)
     

In [26]:
e1=torch.zeros(4,100)
e2=torch.zeros(4,100)
e3=torch.stack([e1,e2],0)
e3.shape

torch.Size([2, 4, 100])

In [27]:
feature_set_MLP=pd.read_csv('Prostrate_MLP_features_ankit.csv')
feature_set_LSTM=pd.read_csv('prostrate_LSTM_features_ankit.csv')
feature_set_MLP.head()

,gene_name,tag_0,tag_1,tag_2,tag_3,tag_4,tag_5,tag_6,tag_7,tag_8,...,tag_90,tag_91,tag_92,tag_93,tag_94,tag_95,tag_96,tag_97,tag_98,tag_99
0,vhl,0.000000,0.000000,0.0,22.241405,0.000000,0.0,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.000000,10.266189
1,smox,0.000000,0.000000,0.0,0.000000,0.000000,0.0,0.000000,0.000000,7.911898,...,0.000000,0.000000,0.0,0.0,0.0,0.000000,0.000000,0.000000,12.720570,0.000000
2,znf148,0.000000,0.000000,0.0,21.678095,0.000000,0.0,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.000000,10.640351
3,map4k2,20.404165,23.852463,0.0,0.000000,43.247234,0.0,31.033705,0.203660,56.684559,...,0.000000,18.108454,0.0,0.0,0.0,28.755461,5.603198,13.697145,63.017044,0.000000
4,mapk4,42.203545,52.076824,0.0,0.000000,89.526337,0.0,67.578827,10.026752,108.900192,...,3.838364,40.393089,0.0,0.0,0.0,57.628872,18.609310,32.895622,120.597061,0.000000


In [28]:
feature_set_MLP.head()

,gene_name,tag_0,tag_1,tag_2,tag_3,tag_4,tag_5,tag_6,tag_7,tag_8,...,tag_90,tag_91,tag_92,tag_93,tag_94,tag_95,tag_96,tag_97,tag_98,tag_99
0,vhl,0.000000,0.000000,0.0,22.241405,0.000000,0.0,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.000000,10.266189
1,smox,0.000000,0.000000,0.0,0.000000,0.000000,0.0,0.000000,0.000000,7.911898,...,0.000000,0.000000,0.0,0.0,0.0,0.000000,0.000000,0.000000,12.720570,0.000000
2,znf148,0.000000,0.000000,0.0,21.678095,0.000000,0.0,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.000000,10.640351
3,map4k2,20.404165,23.852463,0.0,0.000000,43.247234,0.0,31.033705,0.203660,56.684559,...,0.000000,18.108454,0.0,0.0,0.0,28.755461,5.603198,13.697145,63.017044,0.000000
4,mapk4,42.203545,52.076824,0.0,0.000000,89.526337,0.0,67.578827,10.026752,108.900192,...,3.838364,40.393089,0.0,0.0,0.0,57.628872,18.609310,32.895622,120.597061,0.000000


In [29]:
header_of_MLP=['tag_'+str(i) for i in range(feature_set_MLP.shape[1]-1)]
features_MLP=np.array(feature_set_MLP[header_of_MLP])
gene_MLP=feature_set_MLP['gene_name']
print(features_MLP.shape)
print(len(gene_MLP))

(2424, 100)
2424


In [30]:
dictionary_MLP={}
u=0
for gn in gene_MLP:
    dictionary_MLP[gn]=features_MLP[u]
    u=u+1

In [31]:
header_of_LSTM=['tag_'+str(i) for i in range(feature_set_LSTM.shape[1]-1)]
features_LSTM=np.array(feature_set_LSTM[header_of_LSTM])
gene_LSTM=feature_set_LSTM['gene_name']
print(features_LSTM.shape)
print(len(gene_LSTM))

(1763, 100)
1763


In [32]:
dictionary_LSTM={}
u=f=0
for gn in gene_LSTM:
    if gn in dictionary_LSTM.keys():
#         print(gn)
        f=f+1
    dictionary_LSTM[gn]=features_LSTM[u]
    u=u+1
print(f)

107


In [33]:
print(len(dictionary_LSTM.keys()))

1656


In [34]:
fil=open('../../Multi-modality/Model/Prostrate/uni model/MLP/Labels_prostate.txt','r')
tmp=list()
for line in fil:
	tmp.append(int(line))

label_PROSTRATE=np.array(tmp)
print(label_PROSTRATE.shape)

(2424,)


In [47]:
class Sequenceloader(data.Dataset):
    def __init__(self,GN,Feat,label):
        self.gene_names=GN
        self.features_mlp=Feat
        self.label=label
        self.coincdgene_name=[]
        self.coincidfeature_MLP=[]
        self.coincidfeature_LSTM=[]
        self.coincidlabel=[]
        for i in range(len(self.gene_names)):
            u=self.gene_names[i]
            if u in dictionary_LSTM.keys():
                
 
                ch=1   
                for jj in range(ch):
                    self.coincdgene_name.append(u)
                    self.coincidfeature_MLP.append(self.features_mlp[i])
                    self.coincidfeature_LSTM.append(dictionary_LSTM[u])
                    self.coincidlabel.append(self.label[i])
        
        
    
    def __len__(self):
#         print(len(self.dataset))
        return len(self.coincdgene_name)       

    def __getitem__(self, index):
         return np.array(self.coincidfeature_MLP[index]),np.array(self.coincidfeature_LSTM[index]),np.array(self.coincidlabel[index])
#         print(self.dataset['gen_name'][index])          
#         try:
           
#         except :
            
total_set=Sequenceloader(gene_MLP,features_MLP,label_PROSTRATE)  
a=b=c=0
for x,y,z in total_set:
    if(z==0):
        a=a+1
    elif z==1:
        b=b+1
    else:
        c=c+1
print(a,b,c)
a+b    

787 976 0


1763

In [48]:
batch_size = 4
validation_split = .2
shuffle_dataset = True
random_seed= 42

# Creating data indices for training and validation splits:
dataset_size = len(total_set)
indices = list(range(dataset_size))
split = int(np.floor(validation_split * dataset_size))
if shuffle_dataset :
    np.random.seed(random_seed)
    np.random.shuffle(indices)
train_indices, val_indices = indices[split:], indices[:split]

# Creating PT data samplers and loaders:
train_sampler = SubsetRandomSampler(train_indices)
valid_sampler = SubsetRandomSampler(val_indices)

train_loader = torch.utils.data.DataLoader(total_set, batch_size=batch_size, 
                                           sampler=train_sampler)
validation_loader = torch.utils.data.DataLoader(total_set, batch_size=batch_size,
                                                sampler=valid_sampler)


In [49]:
for a,b,c in train_loader:
    print(a.shape,b.shape,c.shape)

torch.Size([4, 100]) torch.Size([4, 100]) torch.Size([4])
torch.Size([4, 100]) torch.Size([4, 100]) torch.Size([4])
torch.Size([4, 100]) torch.Size([4, 100]) torch.Size([4])
torch.Size([4, 100]) torch.Size([4, 100]) torch.Size([4])
torch.Size([4, 100]) torch.Size([4, 100]) torch.Size([4])
torch.Size([4, 100]) torch.Size([4, 100]) torch.Size([4])
torch.Size([4, 100]) torch.Size([4, 100]) torch.Size([4])
torch.Size([4, 100]) torch.Size([4, 100]) torch.Size([4])
torch.Size([4, 100]) torch.Size([4, 100]) torch.Size([4])
torch.Size([4, 100]) torch.Size([4, 100]) torch.Size([4])
torch.Size([4, 100]) torch.Size([4, 100]) torch.Size([4])
torch.Size([4, 100]) torch.Size([4, 100]) torch.Size([4])
torch.Size([4, 100]) torch.Size([4, 100]) torch.Size([4])
torch.Size([4, 100]) torch.Size([4, 100]) torch.Size([4])
torch.Size([4, 100]) torch.Size([4, 100]) torch.Size([4])
torch.Size([4, 100]) torch.Size([4, 100]) torch.Size([4])
torch.Size([4, 100]) torch.Size([4, 100]) torch.Size([4])
torch.Size([4,

torch.Size([4, 100]) torch.Size([4, 100]) torch.Size([4])
torch.Size([4, 100]) torch.Size([4, 100]) torch.Size([4])
torch.Size([4, 100]) torch.Size([4, 100]) torch.Size([4])
torch.Size([4, 100]) torch.Size([4, 100]) torch.Size([4])
torch.Size([4, 100]) torch.Size([4, 100]) torch.Size([4])
torch.Size([4, 100]) torch.Size([4, 100]) torch.Size([4])
torch.Size([4, 100]) torch.Size([4, 100]) torch.Size([4])
torch.Size([4, 100]) torch.Size([4, 100]) torch.Size([4])
torch.Size([3, 100]) torch.Size([3, 100]) torch.Size([3])


In [50]:
def test(test_loader,model):

    total_imgs=0;
    total_corrects=0
    u=0
    nb_classes=2
    confusion_matrix = torch.zeros(nb_classes, nb_classes)
    for i1,i2,label in test_loader:
                

        output=model(i1.to(device).float(),i2.to(device).float())
        total_imgs=total_imgs+label.shape[0]
        z=torch.max(output,1)[1]==label.to(device)
        _, preds = torch.max(output, 1)
#         print(output.shape)
 
        num_corrects=torch.sum(z)
        total_corrects=total_corrects+num_corrects
        for t, p in zip(label.view(-1), preds.view(-1)):
            confusion_matrix[t.long(), p.long()] += 1


        u=u+1
    
    
    print(confusion_matrix)
    return(total_corrects,total_imgs)

In [51]:
device='cuda:7'
model_lstmXMLP=Encoder(2,100,4,300,300,True,4).to(device) #1)mlp 2lstm
uz=torch.rand(4, 100).to(device)
vz=torch.rand(4,100).to(device)
model_lstmXMLP(vz,uz).shape


torch.Size([4, 2])

In [52]:

#model_lstmXMLP.load_state_dict(torch.load(Path('1ANKIT_ILD COMBO_LSTMXMLP.pt')))
optim_params = [
    {'params': model_lstmXMLP.parameters(), 'lr': 0.0001}
]
optimizer = torch.optim.Adam(optim_params)
criterion = nn.CrossEntropyLoss()

In [53]:
epoch=10000
for i in range(epoch):
    torch.save(model_lstmXMLP.state_dict(), '1ANKIT_PROSTATE COMBO_LSTMXMLP.pt')
    print("Accuracy-"+str(test(validation_loader,model_lstmXMLP)))
    total_loss=0
    for inp1,inp2,lab in train_loader:

        

        output=model_lstmXMLP(inp1.to(device).float(),inp2.to(device).float())

        loss_batch=criterion(output,lab.to(device))/4
        optimizer.zero_grad()  
        loss_batch.backward()
        optimizer.step()
        total_loss+=loss_batch
       
    print(total_loss)
    
    
        
        

tensor([[ 75.,  78.],
        [128.,  71.]])
Accuracy-(tensor(146, device='cuda:7'), 352)
tensor(42.0570, device='cuda:7', grad_fn=<AddBackward0>)
tensor([[132.,  21.],
        [ 48., 151.]])
Accuracy-(tensor(283, device='cuda:7'), 352)
tensor(35.7488, device='cuda:7', grad_fn=<AddBackward0>)
tensor([[134.,  19.],
        [ 42., 157.]])
Accuracy-(tensor(291, device='cuda:7'), 352)
tensor(35.3720, device='cuda:7', grad_fn=<AddBackward0>)
tensor([[133.,  20.],
        [ 46., 153.]])
Accuracy-(tensor(286, device='cuda:7'), 352)
tensor(33.8509, device='cuda:7', grad_fn=<AddBackward0>)
tensor([[135.,  18.],
        [ 54., 145.]])
Accuracy-(tensor(280, device='cuda:7'), 352)
tensor(32.9248, device='cuda:7', grad_fn=<AddBackward0>)
tensor([[131.,  22.],
        [ 46., 153.]])
Accuracy-(tensor(284, device='cuda:7'), 352)
tensor(30.5243, device='cuda:7', grad_fn=<AddBackward0>)
tensor([[127.,  26.],
        [ 41., 158.]])
Accuracy-(tensor(285, device='cuda:7'), 352)
tensor(29.6815, device='cuda

tensor([[129.,  24.],
        [ 30., 169.]])
Accuracy-(tensor(298, device='cuda:7'), 352)
tensor(22.7354, device='cuda:7', grad_fn=<AddBackward0>)
tensor([[133.,  20.],
        [ 28., 171.]])
Accuracy-(tensor(304, device='cuda:7'), 352)
tensor(22.8210, device='cuda:7', grad_fn=<AddBackward0>)
tensor([[126.,  27.],
        [ 35., 164.]])
Accuracy-(tensor(290, device='cuda:7'), 352)
tensor(22.9652, device='cuda:7', grad_fn=<AddBackward0>)
tensor([[133.,  20.],
        [ 34., 165.]])
Accuracy-(tensor(298, device='cuda:7'), 352)
tensor(25.0530, device='cuda:7', grad_fn=<AddBackward0>)
tensor([[131.,  22.],
        [ 28., 171.]])
Accuracy-(tensor(302, device='cuda:7'), 352)
tensor(23.1467, device='cuda:7', grad_fn=<AddBackward0>)
tensor([[135.,  18.],
        [ 39., 160.]])
Accuracy-(tensor(295, device='cuda:7'), 352)
tensor(23.9237, device='cuda:7', grad_fn=<AddBackward0>)
tensor([[139.,  14.],
        [ 36., 163.]])
Accuracy-(tensor(302, device='cuda:7'), 352)
tensor(23.8394, device='cuda

tensor([[139.,  14.],
        [ 32., 167.]])
Accuracy-(tensor(306, device='cuda:7'), 352)
tensor(19.8583, device='cuda:7', grad_fn=<AddBackward0>)
tensor([[136.,  17.],
        [ 35., 164.]])
Accuracy-(tensor(300, device='cuda:7'), 352)
tensor(20.9854, device='cuda:7', grad_fn=<AddBackward0>)
tensor([[136.,  17.],
        [ 26., 173.]])
Accuracy-(tensor(309, device='cuda:7'), 352)
tensor(20.6261, device='cuda:7', grad_fn=<AddBackward0>)
tensor([[134.,  19.],
        [ 35., 164.]])
Accuracy-(tensor(298, device='cuda:7'), 352)
tensor(20.3178, device='cuda:7', grad_fn=<AddBackward0>)
tensor([[137.,  16.],
        [ 26., 173.]])
Accuracy-(tensor(310, device='cuda:7'), 352)
tensor(22.1100, device='cuda:7', grad_fn=<AddBackward0>)
tensor([[135.,  18.],
        [ 30., 169.]])
Accuracy-(tensor(304, device='cuda:7'), 352)
tensor(20.7794, device='cuda:7', grad_fn=<AddBackward0>)
tensor([[136.,  17.],
        [ 34., 165.]])
Accuracy-(tensor(301, device='cuda:7'), 352)
tensor(19.2714, device='cuda

tensor([[137.,  16.],
        [ 29., 170.]])
Accuracy-(tensor(307, device='cuda:7'), 352)
tensor(19.5522, device='cuda:7', grad_fn=<AddBackward0>)
tensor([[139.,  14.],
        [ 29., 170.]])
Accuracy-(tensor(309, device='cuda:7'), 352)
tensor(19.0195, device='cuda:7', grad_fn=<AddBackward0>)
tensor([[140.,  13.],
        [ 27., 172.]])
Accuracy-(tensor(312, device='cuda:7'), 352)
tensor(17.5255, device='cuda:7', grad_fn=<AddBackward0>)
tensor([[137.,  16.],
        [ 33., 166.]])
Accuracy-(tensor(303, device='cuda:7'), 352)
tensor(19.1518, device='cuda:7', grad_fn=<AddBackward0>)
tensor([[137.,  16.],
        [ 34., 165.]])
Accuracy-(tensor(302, device='cuda:7'), 352)
tensor(17.7932, device='cuda:7', grad_fn=<AddBackward0>)
tensor([[132.,  21.],
        [ 25., 174.]])
Accuracy-(tensor(306, device='cuda:7'), 352)
tensor(18.5869, device='cuda:7', grad_fn=<AddBackward0>)
tensor([[139.,  14.],
        [ 38., 161.]])
Accuracy-(tensor(300, device='cuda:7'), 352)
tensor(18.5465, device='cuda

tensor([[142.,  11.],
        [ 33., 166.]])
Accuracy-(tensor(308, device='cuda:7'), 352)
tensor(17.8226, device='cuda:7', grad_fn=<AddBackward0>)
tensor([[137.,  16.],
        [ 24., 175.]])
Accuracy-(tensor(312, device='cuda:7'), 352)
tensor(18.4546, device='cuda:7', grad_fn=<AddBackward0>)
tensor([[133.,  20.],
        [ 31., 168.]])
Accuracy-(tensor(301, device='cuda:7'), 352)
tensor(16.6078, device='cuda:7', grad_fn=<AddBackward0>)
tensor([[140.,  13.],
        [ 24., 175.]])
Accuracy-(tensor(315, device='cuda:7'), 352)
tensor(18.1505, device='cuda:7', grad_fn=<AddBackward0>)
tensor([[133.,  20.],
        [ 31., 168.]])
Accuracy-(tensor(301, device='cuda:7'), 352)
tensor(18.5987, device='cuda:7', grad_fn=<AddBackward0>)
tensor([[140.,  13.],
        [ 29., 170.]])
Accuracy-(tensor(310, device='cuda:7'), 352)
tensor(17.8279, device='cuda:7', grad_fn=<AddBackward0>)
tensor([[135.,  18.],
        [ 30., 169.]])
Accuracy-(tensor(304, device='cuda:7'), 352)
tensor(20.0729, device='cuda

...

tensor(11.3874, device='cuda:7', grad_fn=<AddBackward0>)
tensor([[141.,  12.],
        [ 26., 173.]])
Accuracy-(tensor(314, device='cuda:7'), 352)
tensor(10.8251, device='cuda:7', grad_fn=<AddBackward0>)
tensor([[139.,  14.],
        [ 30., 169.]])
Accuracy-(tensor(308, device='cuda:7'), 352)
tensor(11.5814, device='cuda:7', grad_fn=<AddBackward0>)
tensor([[138.,  15.],
        [ 23., 176.]])
Accuracy-(tensor(314, device='cuda:7'), 352)
tensor(11.8642, device='cuda:7', grad_fn=<AddBackward0>)
tensor([[139.,  14.],
        [ 29., 170.]])
Accuracy-(tensor(309, device='cuda:7'), 352)
tensor(12.1686, device='cuda:7', grad_fn=<AddBackward0>)
tensor([[143.,  10.],
        [ 31., 168.]])
Accuracy-(tensor(311, device='cuda:7'), 352)
tensor(12.4531, device='cuda:7', grad_fn=<AddBackward0>)
tensor([[140.,  13.],
        [ 37., 162.]])
Accuracy-(tensor(302, device='cuda:7'), 352)
tensor(12.4256, device='cuda:7', grad_fn=<AddBackward0>)
tensor([[141.,  12.],
        [ 24., 175.]])
Accuracy-(tensor

tensor(12.6229, device='cuda:7', grad_fn=<AddBackward0>)
tensor([[137.,  16.],
        [ 35., 164.]])
Accuracy-(tensor(301, device='cuda:7'), 352)
tensor(11.7830, device='cuda:7', grad_fn=<AddBackward0>)
tensor([[140.,  13.],
        [ 32., 167.]])
Accuracy-(tensor(307, device='cuda:7'), 352)
tensor(10.7887, device='cuda:7', grad_fn=<AddBackward0>)
tensor([[137.,  16.],
        [ 37., 162.]])
Accuracy-(tensor(299, device='cuda:7'), 352)
tensor(12.1718, device='cuda:7', grad_fn=<AddBackward0>)
tensor([[142.,  11.],
        [ 35., 164.]])
Accuracy-(tensor(306, device='cuda:7'), 352)
tensor(11.6549, device='cuda:7', grad_fn=<AddBackward0>)
tensor([[140.,  13.],
        [ 33., 166.]])
Accuracy-(tensor(306, device='cuda:7'), 352)
tensor(10.3958, device='cuda:7', grad_fn=<AddBackward0>)
tensor([[144.,   9.],
        [ 31., 168.]])
Accuracy-(tensor(312, device='cuda:7'), 352)
tensor(12.4368, device='cuda:7', grad_fn=<AddBackward0>)
tensor([[139.,  14.],
        [ 23., 176.]])
Accuracy-(tensor

tensor(12.4553, device='cuda:7', grad_fn=<AddBackward0>)
tensor([[134.,  19.],
        [ 26., 173.]])
Accuracy-(tensor(307, device='cuda:7'), 352)
tensor(12.4693, device='cuda:7', grad_fn=<AddBackward0>)
tensor([[138.,  15.],
        [ 37., 162.]])
Accuracy-(tensor(300, device='cuda:7'), 352)
tensor(14.1688, device='cuda:7', grad_fn=<AddBackward0>)
tensor([[136.,  17.],
        [ 32., 167.]])
Accuracy-(tensor(303, device='cuda:7'), 352)
tensor(12.8443, device='cuda:7', grad_fn=<AddBackward0>)
tensor([[139.,  14.],
        [ 34., 165.]])
Accuracy-(tensor(304, device='cuda:7'), 352)
tensor(12.4361, device='cuda:7', grad_fn=<AddBackward0>)
tensor([[145.,   8.],
        [ 29., 170.]])
Accuracy-(tensor(315, device='cuda:7'), 352)
tensor(12.5938, device='cuda:7', grad_fn=<AddBackward0>)
tensor([[135.,  18.],
        [ 33., 166.]])
Accuracy-(tensor(301, device='cuda:7'), 352)
tensor(12.0504, device='cuda:7', grad_fn=<AddBackward0>)
tensor([[142.,  11.],
        [ 33., 166.]])
Accuracy-(tensor

tensor(13.0373, device='cuda:7', grad_fn=<AddBackward0>)
tensor([[145.,   8.],
        [ 31., 168.]])
Accuracy-(tensor(313, device='cuda:7'), 352)
tensor(12.4781, device='cuda:7', grad_fn=<AddBackward0>)
tensor([[138.,  15.],
        [ 27., 172.]])
Accuracy-(tensor(310, device='cuda:7'), 352)
tensor(13.0458, device='cuda:7', grad_fn=<AddBackward0>)
tensor([[145.,   8.],
        [ 41., 158.]])
Accuracy-(tensor(303, device='cuda:7'), 352)
tensor(13.2183, device='cuda:7', grad_fn=<AddBackward0>)
tensor([[141.,  12.],
        [ 27., 172.]])
Accuracy-(tensor(313, device='cuda:7'), 352)
tensor(10.6005, device='cuda:7', grad_fn=<AddBackward0>)
tensor([[141.,  12.],
        [ 33., 166.]])
Accuracy-(tensor(307, device='cuda:7'), 352)
tensor(10.8850, device='cuda:7', grad_fn=<AddBackward0>)
tensor([[134.,  19.],
        [ 32., 167.]])
Accuracy-(tensor(301, device='cuda:7'), 352)
tensor(11.6723, device='cuda:7', grad_fn=<AddBackward0>)
tensor([[140.,  13.],
        [ 30., 169.]])
Accuracy-(tensor

KeyboardInterrupt: 